In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors


In [4]:
def statistical_outlier_removal(data, k=8, std_dev_multiplier=2.0):
    # Convert DataFrame to numpy array
    points = data[['x', 'y', 'z']].values
    
    # Fit the nearest neighbors model
    nbrs = NearestNeighbors(n_neighbors=k).fit(points)
    distances, _ = nbrs.kneighbors(points)
    
    # Compute mean and standard deviation of distances
    mean_distances = distances.mean(axis=1)
    global_mean = mean_distances.mean()
    global_std_dev = mean_distances.std()
    
    # Determine the threshold
    threshold = global_mean + std_dev_multiplier * global_std_dev
    
    # Identify inliers
    inliers = mean_distances < threshold
    
    # Return filtered data
    return data[inliers]

def save_pcd(data, filename):
    header = (
        "# .PCD v0.7 - Point Cloud Data file format\n"
        "VERSION 0.7\n"
        "FIELDS x y z\n"
        "SIZE 4 4 4\n"
        "TYPE F F F\n"
        "COUNT 1 1 1\n"
        f"WIDTH {len(data)}\n"
        "HEIGHT 1\n"
        "VIEWPOINT 0 0 0 1 0 0 0\n"
        f"POINTS {len(data)}\n"
        "DATA ascii\n"
    )
    
    with open(filename, 'w') as f:
        f.write(header)
        for _, row in data.iterrows():
            f.write(f"{row['x']} {row['y']} {row['z']}\n")
    print(f"PCD file saved to {filename}")

# Load the dataset
input_csv_file = 'synthetic_lidar_data.csv'
data = pd.read_csv(input_csv_file)

# Apply Statistical Outlier Removal
filtered_data = statistical_outlier_removal(data, k=8, std_dev_multiplier=2.0)

# Save the filtered data to a PCD file
output_pcd_file = 'filtered_lidar_data_sor.pcd'
save_pcd(filtered_data, output_pcd_file)

PCD file saved to filtered_lidar_data_sor.pcd
